In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Multiply, Dropout, BatchNormalization, Reshape, Conv2D, MaxPooling2D, Flatten, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

# Define the squeeze-and-excite block
def squeeze_excite_block(input_tensor, ratio=16):
    channels = int(input_tensor.shape[-1])
    x = GlobalAveragePooling2D()(input_tensor)
    x = Dense(channels // ratio, activation='relu')(x)
    x = Dense(channels, activation='sigmoid')(x)
    x = Reshape((1, 1, channels))(x)  # Reshape to match input tensor shape
    x = Multiply()([input_tensor, x])
    return x

# Define custom preprocessing function for Gaussian noise
def apply_gaussian_noise(image):
    noise = np.random.normal(loc=0, scale=0.1, size=image.shape)
    return image + noise

# Set paths
train_dir = "/Users/hiteshwaralavala/Desktop/data set in TTV/train"
test_dir = "/Users/hiteshwaralavala/Desktop/data set in TTV/test"
val_dir = "/Users/hiteshwaralavala/Desktop/data set in TTV/valid"
image_size = (224, 224)  # DenseNet input size
num_classes = 5

# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=apply_gaussian_noise,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Flow from directory with train_data
train_generator = datagen.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical'
)

# Validation and testing data generators
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Load DenseNet model without top layers
base_model = DenseNet121(weights='/Users/hiteshwaralavala/Desktop/data set in TTV/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5', include_top=False, input_shape=(224, 224, 3))

# Add custom top layers with SE block
x = base_model.output
x = squeeze_excite_block(x)  # Add SE block
x = GlobalAveragePooling2D()(x)

# Additional CNN layers
cnn_output = Conv2D(64, kernel_size=(3, 3), activation='relu')(base_model.input)
cnn_output = MaxPooling2D(pool_size=(2, 2))(cnn_output)
cnn_output = Flatten()(cnn_output)  # Add Flatten layer to match shape with GlobalAveragePooling2D output

# Concatenate DenseNet and CNN outputs
concatenated_output = Concatenate()([x, cnn_output])

x = Dense(1024, activation='relu')(concatenated_output)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

# Output layer
predictions = Dense(num_classes, activation='softmax')(x)

# Create final model
model = Model(inputs=[base_model.input], outputs=[predictions])

# Freeze all layers of DenseNet base
for layer in base_model.layers:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)

# Train model with early stopping
history = model.fit(train_generator, epochs=14, validation_data=val_generator, callbacks=[early_stopping])

# Evaluate model on test data
test_loss, test_acc = model.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

# Save model with .h5 extension
model.save('/Users/hiteshwaralavala/Desktop/data set in TTV/hybridcnn hybrid_cnn_with_densenet.h5')


Found 2929 images belonging to 5 classes.
Found 368 images belonging to 5 classes.
Found 365 images belonging to 5 classes.
Epoch 1/14


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


92/92 ━━━━━━━━━━━━━━━━━━━━ 3873s 42s/step - accuracy: 0.4011 - loss: 1.9002 - val_accuracy: 0.5462 - val_loss: 1.8366
Epoch 2/14
92/92 ━━━━━━━━━━━━━━━━━━━━ 3783s 41s/step - accuracy: 0.6195 - loss: 1.2267 - val_accuracy: 0.5245 - val_loss: 1.6656
Epoch 3/14
92/92 ━━━━━━━━━━━━━━━━━━━━ 5098s 56s/step - accuracy: 0.6852 - loss: 0.9159 - val_accuracy: 0.7092 - val_loss: 0.7791
Epoch 4/14
92/92 ━━━━━━━━━━━━━━━━━━━━ 3612s 39s/step - accuracy: 0.6875 - loss: 0.8589 - val_accuracy: 0.4239 - val_loss: 1.3799
Epoch 5/14
92/92 ━━━━━━━━━━━━━━━━━━━━ 9842s 108s/step - accuracy: 0.7054 - loss: 0.8398 - val_accuracy: 0.6386 - val_loss: 1.7923
Epoch 6/14
92/92 ━━━━━━━━━━━━━━━━━━━━ 11613s 127s/step - accuracy: 0.7211 - loss: 0.7634 - val_accuracy: 0.7038 - val_loss: 0.9061
Epoch 7/14
92/92 ━━━━━━━━━━━━━━━━━━━━ 9495s 103s/step - accuracy: 0.7258 - loss: 0.7317 - val_accuracy: 0.7310 - val_loss: 0.9375
12/12 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.6501 - loss: 1.0727


Test Loss: 0.8359920978546143
Test Accuracy: 0.7205479741096497
